# Visual_design_matrix_builder
*Adrien Chopin, 2022*

**This function creates a slack of images (n-by-n pixels nd-array with time as a 3d dimension), generating the visual design of a pRF experiment. The time unit is in TR (half-TR are tolerated). Bars are coded 1 and background 0. Note that the bar will start its pass at the tip of the aperture.**

Inputs are:
- n, the size of the images in px (default: 600x600)
- nTR, the nb of TRs (images) to generate (default: 135)
- bar_list, a list of bar pass directions (default: 1 2 0 3 4 0 5 6 0 7 8 0), coded as follows, with the corresponding angle
    0. Blank pass: do not code the first and last blank delays because we will add some / angle none
    1. LL_UR: from lower left to upper right / angle 45°
    2. Right: from left to right / angle 0°
    3. Up: from down to up / 90°
    4. LR_UL: from lower right to upper left / 135°
    5. UR_LL: from upper right to lower left / 225°
    6. Left: from right to left / 180°
    7. Down: from up to down / 270°
    8. UL_LR: from upper left to lower right / 315°
- pass_duration, the duration of a bar pass in TR (default: 13)
- blank_duration, the duration of a blank pass in TR (default: 6.5)
- delays, a list of the initial and final delays in TR (blanks, default: 5 and 0)
- a_size, circular aperture radius in px - if none, use n (default: 254)
- b_len, the length of the bar in px (default: a_size)
- b_wid, the width of the bar in px (default: 97)

It saves the result in an numpy nd-array and a video (Note: the timing of the video will be incorrect).

## Imports

In [91]:
import numpy as np
import math
import pickle
import cv2
import os

## File and path names

In [92]:
#project_dir = '/scratch/mszinte/data/stereo_prf/'
project_dir = os.path.expanduser('~/disks/meso_S/data/stereo_prf/')
rootpath = os.path.join(project_dir,'derivatives','vdm') # data directory
fileName = 'vdm'
filepath = os.path.join(rootpath,fileName+'.npy')

print(filepath)
print(videopath)

/home/achopin/disks/meso_S/data/stereo_prf/derivatives/vdm/vdm.npy
/home/achopin/disks/meso_S/data/stereo_prf/derivatives/vdm/vdm.mp4


## Default input values

In [93]:
if 'n' not in locals(): n = 600
if 'nTR' not in locals(): nTR = 135
if 'bar_list' not in locals(): bar_list = np.array([1, 2, 0, 3, 4, 0, 5, 6, 0, 7, 8])
if 'pass_duration' not in locals(): pass_duration = 13
if 'blank_duration' not in locals(): blank_duration = 6.5
if 'delays' not in locals(): delays = [5, 6.5]
if 'a_size' not in locals(): a_size = 254
if 'b_len' not in locals(): b_len = a_size
if 'b_wid' not in locals(): b_wid = 97
list_angles = np.array([np.nan, 45, 0, 90, 135, 225, 180, 270, 315])

## Core
### Create a meshgrid of image coordinates x, y, a list of angles by TR


In [94]:
# image coordinates meshgrid
x, y = np.meshgrid(range(0,n), range(0,n))

# calculate the total duration in TR and check that it is equal to nTR
tot = sum(bar_list>0)*pass_duration+sum(bar_list==0)*blank_duration+sum(delays)
if tot!=nTR:
    print(str(tot))
    raise ValueError('something went wrong, total duration is '+str(tot)+' but nb of TR asked is '+str(nTR))
     
# create a list of half TR list with bar pass angles
angle_list = list_angles[bar_list];
angle_halfTR = np.empty((1,2*nTR)); angle_halfTR.fill(np.nan)
head = 0; newhead = 2*delays[0];
angle_halfTR[0,head:newhead]=np.nan
for i in angle_list:
    head = newhead
    if np.isnan(i):
        newhead=int(head+2*blank_duration)
    else:
        newhead=int(head+2*pass_duration) 
    angle_halfTR[0,head:newhead]=i
angle_halfTR[0,newhead:]=np.nan

### Define the function that will draw the frames from the bar position

In [95]:
def draw_frame(x,y,position,n,b_len,b_wid,angle,a_size):
    frame = np.zeros((n,n)) #create n-by-n blank frame
    center_x = round(n/2)
    center_y = round(n/2)
    if ((position[0]-center_x)!=0) & ((position[1]-center_y)!=0):
        position_to_center_line_slope = (position[1]-center_y)/(position[0]-center_x) # slope of the line connecting center of bar and center of screen
        a = -1/position_to_center_line_slope                                          # obtaining the slope of the perpendicular to that line, passing by center of bar (using opposite reciprocal)
        b_low = (position[1]-np.sin(np.radians(angle))*b_wid/2)-a*(position[0]-np.cos(np.radians(angle))*b_wid/2)         # intercept of the line for the lower part of the bar is equal to b = y - ax
        b_up = (position[1]+np.sin(np.radians(angle))*b_wid/2)-a*(position[0]+np.cos(np.radians(angle))*b_wid/2)          # intercept of the line for the upper part of the bar is equal to b = y - ax
        frame[(y>(a*x+min(b_low,b_up)))&(y<(a*x+max(b_low,b_up)))]=1
    else:
        if (position[0]-center_x)==0:
            frame[(y>(position[1]-b_wid/2))&(y<(position[1]+b_wid/2))]=1
        elif (position[1]-center_y)==0:
            frame[(x>(position[0]-b_wid/2))&(x<(position[0]+b_wid/2))]=1
        else:
            print('oops!')
    # apply aperture
    frame[((x-center_x)**2+(y-center_y)**2)>a_size**2]=0
    return frame

### Initialize frames and run through the list of angles to create the frames

In [96]:
#initialization
current_angle = np.nan
center_x = round(n/2)
center_y = round(n/2)
list=np.array([])
# initialize frames with blank frames
frames = np.zeros((n,n,2*nTR));

# main loop
for i in range(0,np.size(angle_halfTR)):
    angle = angle_halfTR[0,i]
    if ~np.isnan(angle): # this is a barpass! (if not, let's keep the blank frame)
        # first check whether it is a new barpass or not
        if angle!=current_angle: # this is a new barpass!
            current_angle = angle
            #starting position for the bar
            start_position = np.array([center_x, center_y])+(a_size+b_wid/2)*np.array([math.cos(math.radians(current_angle+180)),math.sin(math.radians(current_angle+180))]) 
            end_position = np.array([center_x, center_y])+(a_size+b_wid/2)*np.array([math.cos(math.radians(current_angle)),math.sin(math.radians(current_angle))])
            distance = end_position - start_position
            step = distance/(2*pass_duration-1) # n-1 steps
            position = start_position
            list=np.append(list,i)
            print('Done direction angle '+str(angle))
        else:
        # determine the current x,y position of the barpass
            position = position + step    # this one is not rounded to avoid accumulating rounding error
        position_rnd = position.round() 
        frames[:,:,i]=draw_frame(x,y,position_rnd,n,b_len,b_wid,angle,a_size)

# only save the full TR, not the half-TR
frames = frames[:,:,0::2]

Done direction angle 45.0
Done direction angle 0.0
Done direction angle 90.0
Done direction angle 135.0
Done direction angle 225.0
Done direction angle 180.0
Done direction angle 270.0
Done direction angle 315.0


In [97]:
#import matplotlib.pyplot as pyplot
#import matplotlib as plt
#for i in list:
#    fig, ax = pyplot.subplots() 
#    ax.imshow(frames[:,:,int(i)]) 
#for i in range(0,26):
#    fig, ax = pyplot.subplots() 
#    ax.imshow(frames[:,:,int(list[0]+i)]) 
    

In [98]:
#import matplotlib.pyplot as pyplot
#import matplotlib as plt
#fig, ax = pyplot.subplots() 
#position_rnd=[-2.5, 300]
#angle = 0
#ax.imshow(draw_frame(x,y,position_rnd,n,b_len,b_wid,angle,a_size)) 

### Save frames through np.save and export a video

In [99]:
# save numpy array
np.save(filepath,frames)
print('Data saved to '+fileName)

Data saved to vdm


In [100]:
# export a video (NOTE: the timing of the video will be incorrect)
videopath = os.path.join(rootpath,fileName+'.mp4')
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(videopath, fourcc, 1, (n, n), False)
for frame in np.split(frames, nTR, axis=2):
    #frame y needs to be inverted
    frame_inv = frame[-1::-1,:]
    out.write(np.uint8(frame_inv*255))
out.release() 
print('Video conversion done')

Video conversion done
